In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [47]:
# 将建筑和楼层进行合并，形成新的编码


# 1. 读取训练数据（路径请按需修改）
train_df = pd.read_csv('C:/Users/20623/Desktop/project_code/UJIIndoorLoc-PG/UJIIndoorLoc/trainingData.csv')
valid_df = pd.read_csv('C:/Users/20623/Desktop/project_code/UJIIndoorLoc-PG/UJIIndoorLoc/validationData.csv')
train_df_noisy1 = pd.read_csv('../../data/train_noisy1.csv')

# train_df = pd.concat([train_df, train_df_noisy1], ignore_index=True)
# total_df = pd.concat([train_df, valid_df], ignore_index=True)

# 2. 创建联合标签列（如 "2_3" 表示 BUILDINGID=2 且 FLOOR=3）
train_df['location_label'] = train_df['BUILDINGID'].astype(str) + '_' + train_df['FLOOR'].astype(str)
valid_df['location_label'] = valid_df['BUILDINGID'].astype(str) + '_' + valid_df['FLOOR'].astype(str)
# total_df['location_label'] = total_df['BUILDINGID'].astype(str) + '_' + total_df['FLOOR'].astype(str)

# 3. 将联合标签进行整数编码
label_encoder = LabelEncoder()
train_df['location_label_encoded'] = label_encoder.fit_transform(train_df['location_label'])
valid_df['location_label_encoded'] = label_encoder.fit_transform(valid_df['location_label'])
# total_df['location_label_encoded'] = label_encoder.fit_transform(valid_df['location_label'])
total_df = pd.concat([train_df, valid_df])
# 4. 保存为 CSV 文件
# train_df.to_csv('./data/processed_train.csv', index=False)
# valid_df.to_csv('./data/processed_valid.csv', index=False)

# print("✅ 文件已成功保存为 'processed_train.csv'")
# print("✅ 文件已成功保存为 'processed_validat.csv'")

In [48]:
# 取特征和标签
# 训练集
training_data = train_df[train_df.columns[:520]].to_numpy()
training_floors = train_df['location_label_encoded'].to_numpy() # FLOOR LABELS
# 验证集
valid_data = valid_df[valid_df.columns[:520]].to_numpy()
valid_floors = valid_df['location_label_encoded'].to_numpy() # FLOOR LABELS
# 数据总和
total_data = total_df[total_df.columns[:520]].to_numpy()
total_floors = total_df['location_label_encoded'].to_numpy() # FLOOR LABELS

In [49]:
# 数据归一化处理
from Algorithms.utill.data_standar import normalize_rssi, normalize_coords, normalize_test_or_valid_data
# 数据标准化,从总数居获取最大值最小值
X_totalCo_cnn, X_min, X_max = normalize_rssi(total_data)
print(X_min, X_max)
# 训练集特征标准化
training_data = normalize_test_or_valid_data(X_min, X_max, training_data)
# 验证集特征标准化
valid_data = normalize_test_or_valid_data(X_min, X_max, valid_data)
# 全集
total_data = normalize_test_or_valid_data(X_min, X_max, total_data)

-104 100


# PCA方式降维

In [19]:
from sklearn.model_selection import train_test_split

X_trainF, X_testF, y_trainF, y_testF = train_test_split(training_data, training_floors, test_size=0.2, random_state=2812)

In [20]:
# 标准化
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
X_trainF_scaled = standard_scaler.fit_transform(X_trainF)
X_testF_scaled = standard_scaler.fit_transform(X_testF)


In [21]:
# 使用pca进行降维
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
X_trainF_reduced = pca.fit_transform(X_trainF_scaled)
X_testF_reduced = pca.fit_transform(X_testF_scaled)

In [22]:
X_trainF_reduced.shape, X_testF_reduced.shape

((15949, 100), (3988, 100))

# 填补23*23进行CNN

In [55]:
# Augmenting data shape to fit 23x23
training_data_aug = np.empty((len(training_data),529))
for x in range(len(training_data)):
    training_data_aug[x] = np.concatenate((training_data[x], np.full(shape=9,fill_value=1)))
training_data_aug.shape
# training_data_aug = training_data

# Augmenting data shape to fit 23x23
valid_data_aug = np.empty((len(valid_data),529))
for x in range(len(valid_data)):
    valid_data_aug[x] = np.concatenate((valid_data[x], np.full(shape=9,fill_value=1)))
# valid_data_aug.shape
# training_data_aug = training_data
total_data_aug = np.empty((len(total_data),529))
for x in range(len(total_data)):
    total_data_aug[x] = np.concatenate((total_data[x], np.full(shape=9,fill_value=1)))
# valid_data_aug.shape

In [60]:
from sklearn.model_selection import train_test_split

# total_space_ids = total_df['SPACEID'].values
# X_temp, X_testF_cnn, y_temp, y_testF_cnn, stratify_temp, stratify_test= train_test_split(total_data_aug.reshape(len(total_data_aug),23,23), total_floors, test_size=0.2, random_state=2812,stratify=total_space_ids)
# 
# X_trainF_cnn, X_valid_F_cnn, y_trainF_cnn, y_valid_F_cnn = train_test_split(
#     X_temp,
#     y_temp,
#     test_size=1/8,   # 因为 90% * (2/9) ≈ 20%
#     random_state=2812,
#     stratify=stratify_temp
# )

# 准备数据
X = total_data_aug.reshape(len(total_data_aug), 23, 23)
y = total_floors
z = total_df['SPACEID'].to_numpy()  # 分层标签

assert len(X) == len(y) == len(z), (len(X), len(y), len(z))

# 第一步：先切 20% 测试集
X_temp, X_testF_cnn, y_temp, y_testF_cnn, stratify_temp, stratify_test = train_test_split(
    X, y, z,
    test_size=0.2,               # 20% → 测试
    random_state=2812,
    stratify=z
)

# 第二步：在剩下的 80% 里切 10% 验证 (0.8 * 1/8 = 0.1)
X_trainF_cnn, X_valid_F_cnn, y_trainF_cnn, y_valid_F_cnn = train_test_split(
    X_temp, y_temp,
    test_size=1/8,               # 10% → 验证
    random_state=2812,
    stratify=stratify_temp
)
# X_trainF_cnn, X_testF_cnn, y_trainF_cnn, y_testF_cnn = train_test_split(training_data_aug.reshape(len(training_data_aug),23,23), training_floors, test_size=0.2, random_state=2812)

# X_valid_F_cnn, y_valid_F_cnn = valid_data_aug.reshape(len(valid_data_aug),23,23), valid_floors

In [63]:
import numpy as np
import pandas as pd

def check_split_distribution(y_train, y_val, y_test, stratify_label_name="SPACEID"):
    """
    查看训练集/验证集/测试集在标签上的分布
    y_train, y_val, y_test: 标签数组 (如楼层或空间 ID)
    stratify_label_name: 打印时的标签名字，默认是 'SPACEID'
    """
    def _dist(y, name):
        vals, counts = np.unique(y, return_counts=True)
        df = pd.DataFrame({stratify_label_name: vals, "count": counts})
        df["percent"] = df["count"] / df["count"].sum()
        df["set"] = name
        return df

    df_all = pd.concat([
        _dist(y_train, "train"),
        _dist(y_val, "val"),
        _dist(y_test, "test")
    ])
    return df_all.pivot_table(index=stratify_label_name, columns="set", values="percent", fill_value=0)


In [64]:
# 假设分层用的是 SPACEID，那就用同样的标签传进去
dist_table = check_split_distribution(stratify_temp, y_valid_F_cnn, stratify_test, stratify_label_name="SPACEID")

print(dist_table)


set          test     train       val
SPACEID                              
0        0.052732  0.052797  0.056057
1        0.003088  0.003088  0.086461
2        0.003800  0.003801  0.073159
3        0.004513  0.004632  0.076485
4        0.004513  0.004514  0.059857
...           ...       ...       ...
248      0.005226  0.005226  0.000000
249      0.001425  0.001425  0.000000
250      0.000950  0.000950  0.000000
253      0.000950  0.000950  0.000000
254      0.000475  0.000475  0.000000

[124 rows x 3 columns]


## CNN Classifier

In [38]:
X_trainF_reduced.shape, X_testF_reduced.shape

((15949, 100), (3988, 100))

In [39]:
# 计算padding
def compute_padding(size = 23, kernel_size=3, stride=1):
    """
    计算 padding 大小（默认对称填充，适用于 SAME padding）

    参数：
        input_size: 输入特征图尺寸（int）
        output_size: 输出特征图尺寸（int）
        kernel_size: 卷积核大小（int）
        stride: 步长（int）

    返回：
        padding: 应该填充的总大小（int），通常为左右/上下之和
    """
    padding = ((size-1)*stride-size+kernel_size) // 2
    return padding


In [40]:
from Algorithms.CNN import CNNClassifier

In [65]:

# Floor Classifier
dropout = [0.3,0.4,0.5]
lrs = [0.01]
kernel_sizes = [3,5,7]
strides = [1]
model_order = 82
weight_decays = [0.01, 0.001, 0.0001]
for val in dropout:
    for lr in lrs:
        for kernel_size in kernel_sizes:
            for stride in strides:
                for weight_decay in weight_decays:
                    # padding = compute_padding(kernel_size, stride)
                    padding = 0
                    print(f'(Dropout: {val})')
                    cnn = CNNClassifier(n_classes=13, dropout=val, kernel_size=kernel_size, stride=stride, padding=padding) 
                    cnn.fit(X_trainF_cnn, y_trainF_cnn, X_testF_cnn, y_testF_cnn)
                    cnn.train_model(num_epochs=50,eval_train=True, lr=lr, batch_size=64, patience = 7, sava_model_name="./model/"+str(model_order)+"test.pth", kernel_size = kernel_size, stride = stride, dropout=val, weight_decay=weight_decay)
                    model_order += 1

(Dropout: 0.3)
Epoch [1/50], Step [231/231], Loss: 0.8760
Training Accuracy: 68.29566192626953
Test Accuracy: 67.79096984863281
-> Model improved. Saving 
------------------------------
Epoch [2/50], Step [231/231], Loss: 0.5842
Training Accuracy: 79.69184875488281
Test Accuracy: 80.73633575439453
-> Model improved. Saving 
------------------------------
Epoch [3/50], Step [231/231], Loss: 0.6110
Training Accuracy: 79.84796142578125
Test Accuracy: 80.07125854492188
-> No improvement. Patience: 1/7
------------------------------
Epoch [4/50], Step [231/231], Loss: 0.6226
Training Accuracy: 80.74391174316406
Test Accuracy: 80.80760192871094
-> Model improved. Saving 
------------------------------
Epoch [5/50], Step [231/231], Loss: 0.6096
Training Accuracy: 81.30726623535156
Test Accuracy: 81.28265380859375
-> Model improved. Saving 
------------------------------
Epoch [6/50], Step [231/231], Loss: 0.6105
Training Accuracy: 82.32539367675781
Test Accuracy: 81.73396301269531
-> Model im

In [66]:
# 验证模型效果
from Algorithms.CNN import CNNRegressor
import numpy as np
# 假设你的模型是 CNNRegressor，有两个目标值
dropout = [0.3,0.4,0.5]
lrs = [0.01]
kernel_sizes = [3,5,7]
strides = [1]
model_order = 82
weight_decays = [0.01, 0.001, 0.0001]
# 加载保存的权重
for val in dropout:
    for lr in lrs:
        for kernel_size in kernel_sizes:
            for stride in strides:
                for weight_decay in weight_decays:
                    sava_model_name="./model/"+str(model_order)+"test.pth"
                    padding = 0
                    model = CNNClassifier(n_classes=13, kernel_size=kernel_size, stride=stride, padding=padding, dropout=val)
                    model.load_model(sava_model_name)
                    model.X_train = np.zeros_like(X_valid_F_cnn)  # 随便补一个假的训练集占位，不会用到
                    model.Y_train = np.zeros_like(y_valid_F_cnn)
                    # 注意：还需要加载相应的数据
                    model.X_test = X_valid_F_cnn
                    model.Y_test = y_valid_F_cnn
                    model.create_loaders(batch_size=64)
                    correct,_ = model.test(model.loaders['test'])
    
                    print(f"valid collection: {correct} ")
                    model_order += 1

valid collection: 85.03562927246094 
valid collection: 95.39192199707031 
valid collection: 93.77671813964844 
valid collection: 85.41567993164062 
valid collection: 88.59857177734375 
valid collection: 11.923990249633789 
valid collection: 11.923990249633789 
valid collection: 81.85273742675781 
valid collection: 11.923990249633789 
valid collection: 85.74822235107422 
valid collection: 95.96199035644531 
valid collection: 93.39667510986328 
valid collection: 84.1805191040039 
valid collection: 86.08075714111328 
valid collection: 88.12351989746094 
valid collection: 11.923990249633789 
valid collection: 11.923990249633789 
valid collection: 11.923990249633789 
valid collection: 85.03562927246094 
valid collection: 92.87410736083984 
valid collection: 11.923990249633789 
valid collection: 11.923990249633789 
valid collection: 79.80997467041016 
valid collection: 90.97386932373047 
valid collection: 11.923990249633789 
valid collection: 11.923990249633789 
valid collection: 11.92399024